In [88]:
pip install yfinance

In [89]:
pip install ta

Note: you may need to restart the kernel to use updated packages.


In [90]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [91]:
import yfinance as yf
from ta import add_all_ta_features
from ta.utils import dropna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [92]:
def fetch_stock_data(ticker, start_date, end_date):
    try:
        # Fetch historical stock data using yfinance
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        print("Error fetching data:", e)
        return None

In [93]:
def perform_technical_analysis(stock_data):
    try:
        # Reset index
        stock_data.reset_index(inplace=True)

        # Add technical analysis features
        stock_data = dropna(stock_data)
        stock_data = add_all_ta_features(stock_data, open="Open", high="High", low="Low", close="Close", volume="Volume")

        return stock_data
    except Exception as e:
        print("Error performing technical analysis:", e)
        return None


In [94]:
def train_model(X, y):
    # Split features and target into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a machine learning model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Model Accuracy:", accuracy)

    return model, X_train, X_test, y_train, y_test

In [95]:
def predict_movement(model, features):
    # Predict stock movement
    prediction = model.predict([features])

    return prediction

In [96]:
def main():
    # Input stock ticker
    ticker = input("Enter the stock ticker: ")
    print("Fetching data for ticker:", ticker)  # Debugging statement

    # Fetch historical stock data
    start_date = "2019-01-01"  # Change start date as needed
    end_date = "2021-04-01"    # Change end date as needed
    stock_data = fetch_stock_data(ticker, start_date, end_date)

    if stock_data is None:
        print("Failed to fetch stock data for ticker:", ticker)
        return

    print("Fetched data successfully.")  # Debugging statement

    print("Stock data before technical analysis:", stock_data)  # Debugging statement

    # Perform technical analysis
    stock_data = perform_technical_analysis(stock_data)

    if stock_data is None:
        print("Failed to perform technical analysis.")
        return

    print("Performed technical analysis successfully.")  # Debugging statement

    # Prepare features and target
    features = stock_data.drop(['Close', 'Date'], axis=1)
    target = (stock_data['Close'] - stock_data['Open']).apply(lambda x: 1 if x > 0 else 0)

    # Train machine learning model
    model, _, _, _, _ = train_model(features, target)  # Unpack only the model

    # Predict stock movement
    prediction = predict_movement(model, features.iloc[-1])

    # Print prediction
    if prediction[0] == 1:
        print("This stock will rise.")
    else:
        print("This stock will go down.")

if __name__ == "__main__":
    main()

[*********************100%%**********************]  1 of 1 completed

Fetching data for ticker: TSLA
Fetched data successfully.
Stock data before technical analysis:                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-01-02   20.406668   21.008667   19.920000   20.674667   20.674667   
2019-01-03   20.466667   20.626667   19.825333   20.024000   20.024000   
2019-01-04   20.400000   21.200001   20.181999   21.179333   21.179333   
2019-01-07   21.448000   22.449333   21.183332   22.330667   22.330667   
2019-01-08   22.797333   22.934000   21.801332   22.356667   22.356667   
...                ...         ...         ...         ...         ...   
2021-03-25  204.333328  215.166672  203.166672  213.463333  213.463333   
2021-03-26  213.956665  214.606674  199.963333  206.236664  206.236664   
2021-03-29  205.213333  205.493332  198.673340  203.763336  203.763336   
2021-03-30  200.583328  212.553329  197.003326  211.873337  211.873337   
2021-03-31  215.

Performed technical analysis successfully.
Model Accuracy: 0.7543859649122807
This stock will rise.


c:\Users\syed_\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
